In [13]:
import pandas as pd
import numpy as np
import torch

In [4]:
from models.dnn import DNN
from preprocessing.encoding import seqToTensor

%load_ext autoreload
%autoreload 2

# Load Data

In [ ]:
data = pd.read_csv("data/preprocessed-1-dnn.csv")
data

In [ ]:
data['TrainSeq']

In [ ]:
# trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
#                                         download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

# testset = torchvision.datasets.CIFAR10(root='./data', train=False,
#                                        download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [5]:
input_length = len(data['TrainSeq'][0])
encoded = []
symbols = '0123456789,-'

for sequence in data['TrainSeq']:
    flattened = torch.flatten(seqToTensor(sequence, input_length, symbols = symbols))
    encoded.append(flattened)

# Define Model

In [ ]:
''' Define key attributes '''
input_size = encoded[0].shape[0]
output_size = len(symbols) * 9

In [7]:
dnn = DNN(input_size, output_size, seed = 5, hidden_layer_param=[640])

In [11]:
''' Test a Forward pass '''
output = dnn(encoded[0])

tensor([0.1137, 0.1059, 0.1095, 0.1099, 0.1310, 0.1083, 0.1161, 0.1069, 0.0986],
       grad_fn=<ExpBackward>)
tensor([0.1133, 0.1138, 0.1241, 0.1084, 0.1058, 0.1136, 0.1128, 0.0997, 0.1085],
       grad_fn=<ExpBackward>)


In [18]:
torch.sum(output[1], dim=0) # Output sums to 1.0

tensor(1.0000, grad_fn=<SumBackward1>)

# Training

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


def run_training(model, trainingloader, epochs=1)
    for epoch in range(epochs):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            


            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0

    print('Finished Training')

In [ ]:
run_training(model, trainingloader, epochs=1)

# Testing it

In [ ]:
def test_run(model):
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

    return 100 * correct / total